In [6]:
import akshare as ak
import pandas as pd
import numpy as np
from datetime import datetime, timedelta


# 设置显示选项
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

def get_cs_index_stocks():
    """
    获取中证指数成分股（如中证红利 000922）
    兼容 akshare 最新列名
    """
    print("正在获取指数成分股...")
    try:
        df = ak.index_stock_cons_csindex(symbol="932368")  # 中证红利指数
        print("原始列名:", df.columns.tolist())

        # 检查必要列是否存在
        if "成分券代码" not in df.columns or "成分券名称" not in df.columns:
            print("错误：缺少必要列 '成分券代码' 或 '成分券名称'")
            return pd.DataFrame(columns=["stock_code", "stock_name"])

        # 提取所需列并重命名
        df = df[["成分券代码", "成分券名称"]].copy()
        df.columns = ["stock_code", "stock_name"]

        # 去除 .SH / .SZ 后缀
        df["stock_code"] = df["stock_code"].str.replace(r"\.SH|\.SZ", "", regex=True)

        print(f"成功获取 {len(df)} 只成分股")
        return df

    except Exception as e:
        print("获取指数成分股失败:", e)
        return pd.DataFrame(columns=["stock_code", "stock_name"])

def get_low_pb_stocks(stocks_df, top_n=10):
    """
    使用 ak.stock_value_em 获取股票的市净率（PB）数据
    筛选出 PB 最低的 top_n 只股票
    """
    print("正在获取PB数据（使用 ak.stock_value_em）...")
    pb_list = []

    for _, row in stocks_df.iterrows():
        code = row["stock_code"]
        name = row["stock_name"]
        try:
            # 获取估值数据
            df = ak.stock_value_em(symbol=code)
            if df.empty:
                print(f"{code} {name}: 数据为空")
                pb = np.nan
            else:
                # 取最新一条数据（按日期排序，取最后一行）
                latest = df.iloc[-1]
                pb = latest["市净率"]
                if pd.isna(pb) or pb < 0:
                    pb = np.nan  # 剔除无效值
        except Exception as e:
            print(f"获取 {code} {name} PB 失败: {e}")
            pb = np.nan

        pb_list.append({"code": code, "name": name, "pb": pb})

    # 构建结果 DataFrame
    pb_df = pd.DataFrame(pb_list)
    pb_df = pb_df.dropna(subset=["pb"])  # 去除 nan
    pb_df = pb_df[pb_df["pb"] > 0]       # 剔除 <=0 的异常值
    pb_df = pb_df.sort_values("pb").head(top_n).reset_index(drop=True)
    return pb_df
    

def get_low_weekly_return_stocks(stocks_df, top_n=10):
    """
    使用 ak.stock_zh_a_hist(period="weekly", adjust="hfq") 获取后复权周线数据
    开始日期设为 45 天前（确保能获取至少12周数据）
    计算：
      1. 最近一周涨跌幅
      2. 过去12周累计涨跌幅（过滤跌幅 > 30% 的股票）
    返回最近一周涨跌幅最低的 top_n 只股票
    """
    print("正在获取周涨幅数据（后复权，过滤过去12周跌幅>30%）...")

    # 设置开始日期：拉取至少12周数据（约3个月）
    start_date = (datetime.now() - timedelta(days=90)).strftime("%Y%m%d")
    end_date = datetime.now().strftime("%Y%m%d")

    return_list = []

    for _, row in stocks_df.iterrows():
        code = row["stock_code"]
        name = row["stock_name"]
        try:
            # 获取后复权周线数据
            df = ak.stock_zh_a_hist(
                symbol=code,
                period="weekly",
                adjust="hfq",
                start_date=start_date,
                end_date=end_date
            )

            if df.empty or len(df) < 2:
                print(f"{code} {name}: 周线数据不足")
                continue

            # 确保按日期升序排列
            df = df.sort_values("日期").reset_index(drop=True)

            # --- 过滤：过去12周累计跌幅不能超过30% ---
            lookback_weeks = min(12, len(df))  # 最多看12周，如果不足则用全部
            price_12_weeks_ago = df.iloc[-lookback_weeks]["收盘"]
            current_price = df.iloc[-1]["收盘"]
            return_12_weeks = (current_price - price_12_weeks_ago) / price_12_weeks_ago

            if return_12_weeks < -0.3:  # 跌幅 > 30%
                print(f"{code} {name}: 过去{lookback_weeks}周跌幅为 {return_12_weeks:.1%}，已过滤")
                continue

            # --- 计算最近一周涨跌幅 ---
            prev_week = df.iloc[-2]
            curr_week = df.iloc[-1]
            weekly_return = (curr_week["收盘"] - prev_week["收盘"]) / prev_week["收盘"]

            return_list.append({
                "code": code,
                "name": name,
                "weekly_return": weekly_return,
                "12_week_return": return_12_weeks  # 可选：保留长期收益用于分析
            })

        except Exception as e:
            print(f"获取 {code} {name} 数据失败: {e}")
            continue

    # 构建结果 DataFrame
    if not return_list:
        print("没有符合条件的股票。")
        return pd.DataFrame()

    return_df = pd.DataFrame(return_list)
    return_df = return_df.sort_values("weekly_return").head(top_n).reset_index(drop=True)

def main():
    print("策略开始运行...")
    print("当前时间:", datetime.now().strftime("%Y-%m-%d %H:%M:%S"))

    # 1. 获取现金流指数成分股
    stocks_df = get_cs_index_stocks()
    if stocks_df.empty:
        print("成分股获取失败，退出。")
        return

    print(f"共获取成分股 {len(stocks_df)} 只")

    # 2. 获取低PB的10只（底仓，季度调仓）
    low_pb_stocks = get_low_pb_stocks(stocks_df, top_n=10)
    print("\n【底仓 - 低PB前10】")
    print(low_pb_stocks[["code", "name", "pb"]])

    # 3. 获取周涨幅最低的10只（机动仓位，周调仓）
    low_return_stocks = get_low_weekly_return_stocks(stocks_df, top_n=10)
    print("\n【机动仓位 - 周涨幅最低前10】")
    print(low_return_stocks[["code", "name", "weekly_return"]])


if __name__ == "__main__":
    main()

策略开始运行...
当前时间: 2025-08-26 19:54:59
正在获取指数成分股...
原始列名: ['日期', '指数代码', '指数名称', '指数英文名称', '成分券代码', '成分券名称', '成分券英文名称', '交易所', '交易所英文名称']
成功获取 50 只成分股
共获取成分股 50 只
正在获取PB数据（使用 ak.stock_value_em）...

【底仓 - 低PB前10】
     code  name        pb
0  600585  海螺水泥  0.693070
1  000039  中集集团  0.926509
2  600219  南山铝业  0.976356
3  601880  辽港股份  0.980809
4  601919  中远海控  1.012625
5  600352  浙江龙盛  1.021228
6  002120  韵达股份  1.221297
7  601877  正泰电器  1.268501
8  600295  鄂尔多斯  1.341260
9  600482  中国动力  1.355953
正在获取周涨幅数据（后复权，近10天）...

【机动仓位 - 周涨幅最低前10】
     code  name  weekly_return
0  603129  春风动力      -0.038927
1  603939  益丰药房      -0.018418
2  603233   大参林      -0.006956
3  601225  陕西煤业      -0.003954
4  600380   健康元      -0.003546
5  002120  韵达股份      -0.002054
6  300002  神州泰岳      -0.000640
7  603885  吉祥航空       0.002049
8  601880  辽港股份       0.006623
9  002056  横店东磁       0.007829

结果已保存至: 现金流策略_20250826.xlsx


In [1]:
import akshare as ak

stock_individual_info_em_df = ak.stock_individual_info_em(symbol="002056")
print(stock_individual_info_em_df)
ak.stock_board_industry_name_em() 

/home/jovyan/data/.venv/lib/python3.11/site-packages/py_mini_racer/py_mini_racer.py:15: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


   item               value
0    最新               18.45
1  股票代码              002056
2  股票简称                横店东磁
3   总股本        1626712074.0
4   流通股        1625031609.0
5   总市值  30012837765.299999
6  流通市值  29981833186.049999
7    行业                光伏设备
8  上市时间            20060802
